### 目標: 找出 ```AAPL``` 跟 ```MSFT``` 的 balance sheet, income statement, cash flow statement, 與 beta 值:

拆分目標:  
- 先創一個 dictionary  
- 因為有多個股票，所以要迴圈  
    - 迴圈內要有另一個 dictionary，用來放各種資訊 
    - 迴圈的最後，要把 local 的 dictionary 裡的資訊放入 global 的 dictionary(也就是巢狀 dictionary)  
- 將巢狀迴圈轉型成 ```pandas``` 的 ```Data Frame```  
- 將 ```NaN``` 刪除

In [6]:
import requests 
import pandas as pd 
from bs4 import BeautifulSoup

In [7]:
financial_dict = {}
tickers = ["AAPL", "MSFT"]

In [8]:
for ticker in tickers:
    # get balance sheet information
    temp_dict = {}
    url = "https://finance.yahoo.com/quote/"+ticker+"/balance-sheet?p="+ticker 
    page = requests.get(url)
    page_content = page.content # getting html code
    soup = BeautifulSoup(page_content, "html") # getting Beautiful soup instance
    temp = soup.find("div", {"class":"D(tbrg)"}) 
    rows = temp.find_all("div", {"data-test":"fin-row"})
    for row in rows:
        title = row.find("span", {"class":"Va(m)"})
        title = title.get_text() # 取得科目名稱
        num = row.find("div", {"data-test":"fin-col"}) 
        num = num.get_text() # 取得科目數據
        temp_dict[title] = num

    # get income statement information
    url = "https://finance.yahoo.com/quote/"+ticker+"/financials?p="+ticker
    page = requests.get(url)
    page_content = page.content # getting html code
    soup = BeautifulSoup(page_content, "html") # gettting Beautiful soup instance
    temp = soup.find("div", {"class":"D(tbrg)"}) 
    rows = temp.find_all("div", {"data-test":"fin-row"})
    for row in rows:
        title = row.find("span", {"class":"Va(m)"})
        title = title.get_text() # 取得科目名稱
        num = row.find("div", {"data-test":"fin-col"}) 
        num = num.get_text() # 取得科目數據
        temp_dict[title] = num

    # get cash flow information
    url = "https://finance.yahoo.com/quote/"+ticker+"/cash-flow?p="+ticker
    page = requests.get(url)
    page_content = page.content # getting html code
    soup = BeautifulSoup(page_content, "html") # getting Beautiful soup instance
    temp = soup.find("div", {"class":"D(tbrg)"}) 
    rows = temp.find_all("div", {"data-test":"fin-row"})
    for row in rows:
        title = row.find("span", {"class":"Va(m)"})
        title = title.get_text() # 取得科目名稱
        num = row.find("div", {"data-test":"fin-col"}) 
        num = num.get_text() # 取得科目數據
        temp_dict[title] = num
    
    # get beta information
    url = "https://finance.yahoo.com/quote/"+ticker+"?p="+ticker
    page = requests.get(url)
    page_content = page.content
    soup = BeautifulSoup(page_content, "html")
    temp = soup.find("div", {"data-test":"quote-summary-stats"})
    temp = temp.find_all("tr", {"class":"Bxz(bb) Bdbw(1px) Bdbs(s) Bdc($seperatorColor) H(36px)"})[8]
    title = temp.find("td", {"class":"C($primaryColor) W(51%)"})
    title = title.get_text()
    num = temp.find("td", {"class", "Ta(end) Fw(600) Lh(14px)"})
    num = num.get_text()
    temp_dict[title] = num
    

    financial_dict[ticker] = temp_dict # 將最大的 financial_dict 裡的 key 值設為各 ticker，value 則是那個 ticker 裡的財務資訊

combined_financials = pd.DataFrame(financial_dict)
combined_financials.dropna(axis=0, inplace=True)
tickers = combined_financials.columns
print(combined_financials)
    

                                                           AAPL         MSFT
Total Assets                                        323,888,000  301,311,000
Total Liabilities Net Minority Interest             258,549,000  183,007,000
Total Equity Gross Minority Interest                 65,339,000  118,304,000
Total Capitalization                                164,006,000  177,882,000
Common Stock Equity                                  65,339,000  118,304,000
Net Tangible Assets                                  65,339,000   67,915,000
Working Capital                                      38,321,000  109,605,000
Invested Capital                                    177,775,000  181,631,000
Tangible Book Value                                  65,339,000   67,915,000
Total Debt                                          112,436,000   70,998,000
Net Debt                                             74,420,000   49,751,000
Share Issued                                         16,976,763    7,571,000

我是用 ```.find()```(而非 ```.find_all()```) 來找 ```"fin-col"``` 的數據，如此一來，就可以只取得最新的數據  
得注意到可能有很多不同的數據都是同一個 ```class```，這時就要用 ```.find_all()``` 並用 list 取值的方式來取得你要的數據是位於第幾個 ```class```。

In [10]:
combined_financials

,AAPL,MSFT
Cash and cash equivalents,"3,80,16,000","1,35,76,000"
Other short-term investments,"5,29,27,000","12,29,51,000"
Total cash,"9,09,43,000","13,65,27,000"
Net receivables,"1,61,20,000","3,20,11,000"
Inventory,"-7,75,000","-79,000"
...,...,...
Cash at end of period,"3,50,39,000","1,73,80,000"
Free cash flow,"7,17,06,000","4,92,29,000"
Operating cash flow,"8,00,08,000","6,61,92,000"
Capital expenditure,"-83,02,000","-1,69,63,000"
